# Plot and analyze results of MESA Type I X-ray burst simulations 

#### Read comments at the beginning of each cell to understand what it does

In [1]:
# on an astrohub server use ipympl that enables the interactive features of 
# matplotlib in the Jupyter notebook and in JupyterLab
%pylab ipympl  

# for jupyter classic notebook use
#%pylab nbagg

from nugridpy import mesa as ms
from nugridpy import utils as ut

# begin counting figures
ifig=0
for i in range(0,10000):
    close(i)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#### this cell defines functions that allow to suppress unnecessary output information
import os
from contextlib import contextmanager
@contextmanager
def redirect_stdout(new_target):
    old_target, sys.stdout = sys.stdout, new_target
    try:
        yield new_target
    finally:
        sys.stdout = old_target
def get_devnull():
    #return open(os.devnull, "w")
    return open('log_stuff.txt', "w") #where all the stuff goes you don't want to see
####

In [3]:
# astronomical and physical constants in SI units
from astropy import constants as const
from astropy import units as u

Msun = (const.M_sun).value
Rsun = (const.R_sun).value
Lsun = (const.L_sun).value
GN = (const.G).value  # Newton's constant 
sigma = (const.sigma_sb).value  # 1e3 tarnsforms SI units to sgs units
print ('Msun =',Msun)
print ('Rsun =',Rsun)
print ('Lsun =',Lsun)
print ("Newton's G =",GN)
print ("Stefan-Boltzmann constant =",sigma)

# transform to sgs units
sigma = 1e3*sigma
Rsun = 1e2*Rsun
Msun = 1e3*Msun
Lsun = 1e7*Lsun

print ('\nStefan-Boltzmann sigma and solar radius and mass in cgs units:',sigma,Rsun,Msun,Lsun)

Msun = 1.988409870698051e+30
Rsun = 695700000.0
Lsun = 3.828e+26
Newton's G = 6.6743e-11
Stefan-Boltzmann constant = 5.6703744191844314e-08

Stefan-Boltzmann sigma and solar radius and mass in cgs units: 5.6703744191844314e-05 69570000000.0 1.988409870698051e+33 3.828e+33


In [4]:
# name of your directory on astrohub/outreach server
name = 'dpa' # 'Pavel'

# path to MESA work directory
# on astrohub/csa server
mesa_work_dir = '/user/scratch14_wendi3/'+name+'/canpan_projects/xrb_rp_process/xrb_mesa_canpan/' 

# on astrohub/outreach server

# extension of directory name LOGS
suffix = ''

In [5]:
# plot X-ray burst luminosity
sh = ms.history_data(mesa_work_dir+'LOGS'+suffix,clean_starlog=True)
age_s = 365.2422*86400*sh.get('star_age')
age_hr = age_s/3600.
mass_acc = sh.get('star_mass') - 1.4
model = sh.get('model_number')
L38 = (1e-38*Lsun)*(10.**sh.get('log_L'))
lgTeff = sh.get('log_Teff')

Requested new history.datasa; create new from history.data
 reading ...100% 



In [6]:
# HRD of NS experiencing XRBs
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=6
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.6*size)+'in'   # to adjust horizontal figure size

sh.hrd()
tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# select models within all XRB peaks, i.e. where L38 > L38_min
L38_min = 0.02
xrb = -1
i1 = []
i2 = []

# the first MESA model was
model_1 = 3100

for i in range(len(L38)):
    if L38[i] > L38_min and xrb < 0:
        i1.append(i)
        xrb = +1
    if L38[i] < L38_min and xrb > 0:
        i2.append(i)
        xrb = -1
        
k_end = 0
if xrb > 0:
    k_end = 1
for k in range(len(i1)-k_end):
    i22 = -1
    if k <= len(i1)-1:
        i22 = i2[k]
    print ('peak #',k,'from model',i1[k],'to model',i2[k],i22)

peak # 0 from model 32 to model 228 228


In [8]:
# plot all XRBs on a same time scale
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=6
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.6*size)+'in'   # to adjust horizontal figure size

j = 1
for k in range(len(i1)-k_end):
    i22 = -1
    if k <= len(i1)-1:
        i22 = i2[k]
    plot(age_s[i1[k]:i22]-age_s[i1[k]],L38[i1[k]:i22],color=ut.linestylecb(j)[2],linestyle=ut.linestylecb(j)[0],label='XRB #'+str(k+1))
    j += 1

xlim(0,150)
ylim(-0.25,3.5)
xlabel('$\mathrm{time\ (s)}$',fontsize=14)
ylabel('$L_B\ (10^{38}\ \mathrm{erg\,s}^{-1})$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=1,fontsize=10)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# plot XRB luminosity as a function of time in hours
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=6
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.6*size)+'in'   # to adjust horizontal figure size

j = 0
plot(age_hr,L38,color=ut.linestylecb(j)[2],linestyle=ut.linestylecb(j)[0],label='')

#xlim(0,150)
ylim(-0.25,3.5)
xlabel('$\mathrm{time\ (hr)}$',fontsize=14)
ylabel('$L_B\ (10^{38}\ \mathrm{erg\,s}^{-1})$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
#legend(frameon=False,loc=1,fontsize=12)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# plot XRB luminosity as function of model number
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=6
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.6*size)+'in'   # to adjust horizontal figure size

j = 1
plot(model,L38,color=ut.linestylecb(j+7)[2],linestyle=ut.linestylecb(j+7)[0],label='')

#xlim(0,500)
ylim(0,3.5)
xlabel('$\mathrm{model}$',fontsize=14)
ylabel('$L_B\ (10^{38}\ \mathrm{erg\,s}^{-1})$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
#legend(frameon=False,loc=2)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# read profiles.index file from path to LOGS directory that contains results of XRB computations
f = open(mesa_work_dir+'LOGS'+suffix+'/profiles.index', 'r')

profiles = []

i=0
for line in f:
    if i >= 1:
        profiles.append(int(float(line.split()[0])))
    i += 1
    
f.close()

print ("There are",len(profiles),"profiles for the following models:\n", profiles)

There are 23 profiles for the following models:
 [3110, 3120, 3130, 3140, 3150, 3160, 3170, 3180, 3190, 3200, 3210, 3220, 3230, 3240, 3250, 3260, 3270, 3280, 3290, 3300, 3310, 3320, 3330]


In [12]:
# plot a selected XRB along with available profiles

# select k = XRB # - 1 
k = 0

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.4*size)+'in'   # to adjust horizontal figure size

# add models with selected profiles to the above HRD
lgL_plot = []
age_plot = []
prof_plot = []

i22 = -1
if k <= len(i1)-1:
    i22 = i2[k]

j = k+1
plot(age_s[i1[k]:i22]-age_s[i1[k]],L38[i1[k]:i22],color=ut.linestylecb(j)[2],linestyle=ut.linestylecb(j)[0],label='XRB #'+str(k+1))


j = 0
for mod in profiles:
    for i in range(i1[k],i22):
        #print (mod,i)
        if i+model_1 == mod:
            lgL_plot.append(L38[i])
            age_plot.append(age_s[i]-age_s[i1[k]])
            prof_plot.append(mod)
            plot(age_plot[j],lgL_plot[j],marker='o',markerfacecolor=ut.linestylecb(j)[2],\
                markeredgecolor=ut.linestylecb(j)[2],label='model '+str(mod))
            j += 1
            if j > 20:
                j = 0

                
xlim(0,150)
ylim(-0.25,3.0)
xlabel('$\mathrm{time\ (s)}$',fontsize=14)
ylabel('$L_B\ (10^{38}\ \mathrm{erg\,s}^{-1})$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=1,fontsize=10,ncol=4)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# plot mass accreted by NS
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=6
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.6*size)+'in'   # to adjust horizontal figure size

j = 1
semilogy(age_hr,mass_acc,color=ut.linestylecb(j+7)[2],linestyle=ut.linestylecb(j+7)[0],label='')

#xlim(0,500)
#ylim(0,3.0)
xlabel('$\mathrm{time\ (hr)}$',fontsize=14)
ylabel('$M_\mathrm{acc}/M_\odot$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
#legend(frameon=False,loc=2)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# plot radial temperature profiles for the selected XRB peak
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

results_dir = mesa_work_dir + 'LOGS'+suffix

j = 0

with get_devnull() as devnull, redirect_stdout(devnull):
    for model_plot in prof_plot:
        mod=ms.mesa_profile(results_dir,model_plot)
        rad = Rsun*mod.get('radius') # radius in cm
        rad = rad - rad[-1] # radius in cm counted from the envelope base
        lgT = mod.get('logT')

        plot(rad,lgT,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='model '+str(model_plot))
    
        j += 1

xlim(0,5500)
ylim(7,9.25)
xlabel('$r\ \mathrm{(cm)}$',fontsize=14)
ylabel('$\log_{10}\,T\ \mathrm{(K)}$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=1,fontsize=8)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# plot radial density profiles for the selected XRB peak
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

results_dir = mesa_work_dir + 'LOGS'+suffix

j = 0

with get_devnull() as devnull, redirect_stdout(devnull):
    for model_plot in prof_plot:
        mod=ms.mesa_profile(results_dir,model_plot)
        rad = Rsun*mod.get('radius') # radius in cm
        rad = rad - rad[-1] # radius in cm counted from the envelope base
        lgRho = mod.get('logRho')

        plot(rad,lgRho,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='model '+str(model_plot))
    
        j += 1

xlim(0,5500)
ylim(0,7)
xlabel('$r\ \mathrm{(cm)}$',fontsize=14)
ylabel('$\log_{10}\,\\rho\ \mathrm{(K)}$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=1,fontsize=8)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# plot radial profiles of elemental abundances

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 2

model_plot = prof_plot[4]

results_dir = mesa_work_dir + 'LOGS'+suffix
mod=ms.mesa_profile(results_dir,model_plot)
rad = Rsun*mod.get('radius') # radius in cm
rad = rad - rad[-1] # radius in cm counted from the envelope base
xh = mod.get('h1')
xhe = mod.get('he4')
xc = mod.get('c12')
xo = mod.get('o16')
xfe = mod.get('fe56')

semilogy(rad,xh,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='$X(\mathrm{H})$')
semilogy(rad,xhe,color=ut.linestylecb(j+2)[2],linestyle=ut.linestylecb(j+2)[0],label='$X(^4\mathrm{He})$')
semilogy(rad,xc,color=ut.linestylecb(j+3)[2],linestyle=ut.linestylecb(j+3)[0],label='$X(^{12}\mathrm{C})$')
semilogy(rad,xo,color=ut.linestylecb(j+4)[2],linestyle=ut.linestylecb(j+4)[0],label='$X(^{16}\mathrm{O})$')
semilogy(rad,xfe,color=ut.linestylecb(j+5)[2],linestyle=ut.linestylecb(j+5)[0],label='$X(^{56}\mathrm{Fe})$')

xlim(0,5500)
ylim(1e-5,1e0)
xlabel('$r\ \mathrm{(cm)}$',fontsize=14)
ylabel('$\log_{10}\,X_i$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,fontsize=12)
title('model '+str(model_plot))
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

23 in profiles.index file ...
Found and load nearest profile for cycle 3180
reading profile/user/scratch14_wendi3/dpa/canpan_projects/xrb_rp_process/xrb_mesa_canpan/LOGS/profile8.data ...
 reading ...100% 

Closing profile tool ...


In [17]:
# plot profiles of MLT diffusion coefficient for the selected XRB peak
ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

results_dir = mesa_work_dir + 'LOGS'+suffix

j = 0

with get_devnull() as devnull, redirect_stdout(devnull):
    for model_plot in prof_plot:
        mod=ms.mesa_profile(results_dir,model_plot)
        rad = Rsun*mod.get('radius') # radius in cm
        rad = rad - rad[-1] # radius in cm counted from the envelope base
        lgDmix = mod.get('log_mlt_D_mix')

        plot(rad,lgDmix,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='model '+str(model_plot))
    
        j += 1

xlim(0,5500)
ylim(4,10)
xlabel('$r\ \mathrm{(cm)}$',fontsize=14)
ylabel('$\log_{10}\,D_\mathrm{conv}$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=1,fontsize=8)
tight_layout()
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# read se data from MESA XRB simulation
from nugridpy import nugridse as nuse

s=nuse.se(mesa_work_dir+'ns_star_hdf'+suffix)

Searching files, please wait.......
Writing preprocessor files
ns_star_hdf.0003101.se.h5


Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/nugridpy/h5T.py", line 458, in run
    write(self.preprocName,header,dcols,data,sldir=self.filename)
  File "/usr/local/lib/python3.6/dist-packages/nugridpy/ascii_table.py", line 480, in write
    tmp1=data_fmt.format(data[i][j])
ValueError: Unknown format code 'f' for object of type 'str'



In [19]:
# plot and write maximum temperature and density as functions of time for slected XRB

# select k = XRB # -1
k = 0
    
ncyc = i2[k] - i1[k]
    
# find maximum temperature and its corresponding density for all cycles (it may take a few minutes)
t9_max = linspace(0,0,ncyc)
rho_max = linspace(0,0,ncyc)
age_max = linspace(0,0,ncyc)

i = 0
for cyc in range(i1[k],i2[k]):
    cycle = cyc + model_1
    age = s.get(cycle,'age')
    tem = s.get(cycle,'temperature')
    rho = s.get(cycle,'rho')
    item_max = argmax(tem)
    t9_max[i] = tem[item_max] # max(tem)
    rho_max[i] = rho[item_max]
    age_max[i] = age
    i += 1
    #print (len(rho),item_max)

 reading ['rho']...100%]...100%

In [20]:
# find a cycle with maximum T and its age, use this age as age zero-point
cyc_t9max = i1[k] + argmax(t9_max)
print ('\nMaximum temperature T9 =',1e-9*max(t9_max),' (GK) is reached at cycle',cyc_t9max)


Maximum temperature T9 = 1.085666035404317  (GK) is reached at cycle 60


In [21]:
# read in the trajectory from the NuPPN example ppn_XRB_K04

example_dir="/user/scratch14_wendi3/dpa/nuppn/examples/ppn_XRB_K04/"
file_name='trajectory.input'

file_to_read = example_dir+file_name

f = open(file_to_read, 'r')

age_ex=[]
t9_ex = []
rho_ex = []

k = 0

for line in f:
    #print (line)
    if k < 7:
        k += 1
        continue
    #print (line)
    age_ex.append(float(line.split()[0]))
    t9_ex.append(1e9*float(line.split()[1]))
    rho_ex.append(float(line.split()[2]))
    
f.close()
 
n_ex = len(age_ex)
print (n_ex)
#for i in range(n_ex):
#    print (i, age_ex[i],t9_ex[i],rho_ex[i])

2000


In [22]:
# populate the interactive namespace with the function 
# that makes 1d interpolation
from scipy.interpolate import interp1d

# interpolate t9_max and rho_max onto a denser time mesh
n_int = 2000
age_int = linspace(age_max[0],age_max[-1],n_int)
t_int = linspace(0,0,n_int)
rho_int = linspace(0,0,n_int)

kind_int = 'slinear'

f_t = interp1d(age_max,t9_max,kind=kind_int)
f_rho = interp1d(age_max,rho_max,kind=kind_int)

for i in range(n_int):
    t_int[i] = f_t([age_int[i]])
    rho_int[i] = f_rho([age_int[i]])

In [23]:
# plot evolutionary profiles of maximum T and rho
ifig = ifig+1

fig, ax1 = subplots()

size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

name1 = '$T\ \mathrm{(model)}$'
#lns1 = ax1.plot(age_max-age_max[0], t9_max,\
#                color=ut.linestylecb(0)[2], linestyle=ut.linestylecb(0)[0],label=name1)
lns1 = ax1.plot(age_int-age_int[0], t_int,\
                color=ut.linestylecb(0)[2], linestyle=ut.linestylecb(0)[0],label=name1)
name12 = '$T\ \mathrm{(XRB\ example)}$'
lns12 = ax1.plot(age_ex,t9_ex,color=ut.linestylecb(2)[2], linestyle=ut.linestylecb(2)[0],label=name12)

ax1.set_xlabel('time (s)')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel(name1)
ax1.tick_params('y')

#age_minute_max = 2750.
#ax1.set_xlim(-50,age_minute_max)

#t9_max_min = 0.075
#ax1.set_ylim(t9_max_min,0.350)

ax2 = ax1.twinx()
name2 = '$\\rho$'
#lns2 = ax2.semilogy(age_max-age_max[0], rho_max,\
#            color=ut.linestylecb(1)[2],linestyle=ut.linestylecb(1)[0],label=name2)
lns2 = ax2.semilogy(age_int-age_int[0], rho_int,\
            color=ut.linestylecb(1)[2],linestyle=ut.linestylecb(1)[0],label=name2)
name22 = '$\\rho\ (XRB\ example)$'
lns22 = ax2.semilogy(age_ex,rho_ex,color=ut.linestylecb(3)[2],linestyle=ut.linestylecb(3)[0],label=name22)
ax2.set_ylabel(name2+'$\ [\mathrm{g\,cm}^{-3}]$')
ax2.tick_params('y')
#ax2.set_ylim(5e0,1e4)

# added these four lines
#lns = lns1+lns2
lns = lns1+lns12+lns2+lns22
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=7, frameon=False)

fig.tight_layout()
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# save evolutonary changes of maximum T and rho in trajectory file that can be used by ppn code in one-zone simulations
file_out="trajectory.input"
fout=open(mesa_work_dir+file_out,'w')

line1_out="# time T rho\n"
line2_out="# YRS/SEC; T8K/T9K; CGS/LOG\n"
line3_out="# FORMAT: '(10x,A3)'\n"
line4_out="AGEUNIT = SEC\n"
line5_out="TUNIT   = T9K\n"
line6_out="RHOUNIT = CGS\n"
line7_out="ID = 0\n"

fout.write(line1_out)
fout.write(line2_out)
fout.write(line3_out)
fout.write(line4_out)
fout.write(line5_out)
fout.write(line6_out)
fout.write(line7_out)

for i in range(len(age_int)):
    line_out=" "+str("{:.6}".format((age_int[i]-age_int[0])))+" "+str("{:.4}".format(1e-9*t_int[i]))\
        +" "+str("{:.6}".format(rho_int[i]))+"\n"
    #line_out=" "+str(time[i])+" "+str(tem[i])+" "+str(den[i])+" "+str(xh[i])+"\n"  # with X
    fout.write(line_out)
fout.close()